# Aya 23 (8B) Multilingual Polarization Detection
**Powered by Unsloth (2x Faster, 70% Less Memory)**

This notebook fine-tunes the **Aya 23 8B** model (optimized for multilingual tasks like Amharic, Hausa, etc.) using the Unsloth library.

### **Method: Generative Classification**
Instead of adding a classifier head, we train the LLM to output the label as text.
- **Input:** "Analyze this text for polarization: [Text]"
- **Output:** "Polarized" or "Not Polarized" (mapped to 1 and 0 for submission)

**Note:** Ensure your Runtime is set to **T4 GPU**.

In [1]:
%%capture
# 1. Install Unsloth and dependencies
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install unsloth first
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install pandas scikit-learn tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# --- CONFIGURATION ---
# Update these paths to match your Google Drive structure
train_dir = '/content/drive/MyDrive/subtask1/train/'
test_dir = '/content/drive/MyDrive/subtask1/dev/'

# Model Selection
model_name = "unsloth/Qwen2.5-7B-Instruct" # Best for multilingual (Amharic, Hausa, etc.)
max_seq_length = 2048
load_in_4bit = True

In [4]:
import os
import glob
import pandas as pd
from datasets import Dataset

# 1. Load Data
def load_data(directory):
    files = glob.glob(os.path.join(directory, '*.csv'))
    dfs = []
    for file in files:
        df = pd.read_csv(file)
        lang = os.path.basename(file).split('.')[0]
        # Handle variations like 'pred_eng.csv' or just 'eng.csv'
        if lang.startswith('pred_'):
            lang = lang.replace('pred_', '')
        df['language'] = lang
        dfs.append(df)
    if not dfs:
        raise ValueError(f"No CSV files found in {directory}")
    return pd.concat(dfs, ignore_index=True)

print("Loading training data...")
train_df_full = load_data(train_dir)
print(f"Loaded {len(train_df_full)} training examples.")

# 2. Map Labels to Text
# We convert numeric labels (0/1) to text for the LLM to learn.
label_map = {
    0: "Not Polarized",
    1: "Polarized",
    "0": "Not Polarized",
    "1": "Polarized"
}

# Apply mapping
if 'polarization' in train_df_full.columns:
    train_df_full['label_text'] = train_df_full['polarization'].map(label_map)
    train_df_full = train_df_full.dropna(subset=['label_text'])
else:
    raise ValueError("Column 'polarization' not found in training data")

# 3. Format for LLM (Prompt Engineering)
prompt_template = """You are an expert linguist. Analyze the following text for polarization.
Text: {}
Language: {}
Label: {}"""

def format_row(row):
    return prompt_template.format(row['text'], row['language'], row['label_text'])

train_df_full['text_prompt'] = train_df_full.apply(format_row, axis=1)

# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(train_df_full[['text_prompt']])
print("Data formatted for Unsloth.")
print("Example Prompt:\n", dataset[0]['text_prompt'])

Loading training data...
Loaded 40395 training examples.
Data formatted for Unsloth.
Example Prompt:
 You are an expert linguist. Analyze the following text for polarization.
Text: न्याय का लागि लडिरहेका आदिवासी जनजाति आन्दोलनको दौडानमा उत्पन्न हुन सक्ने जुनसुकै घटना र परिणामको जिम्मेवार स्वयम् नेपाल सरकार हुनेछ।
Language: nep
Label: Not Polarized


In [5]:
from unsloth import FastLanguageModel
import torch

# 4. Load Model with Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = None, # Auto detect
    load_in_4bit = load_in_4bit,
    device_map = {"": 0},
)

# 5. Add LoRA Adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth 2025.11.3 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback
from datasets import Dataset

# --- CRITICAL DATA RESET ---\n
dataset = Dataset.from_pandas(train_df_full[['text_prompt']])

# 1. Split Data (Train vs Val)
# First, split 10% for validation so we have a holdout set
dataset_split = dataset.train_test_split(test_size=0.1)
train_dataset = dataset_split['train']

# FIX: Limit Eval Set Size
# Instead of evaluating on the full 10% (2000+ samples), we select just 40 samples.
# This makes the evaluation step extremely fast.
eval_dataset = dataset_split['test'].select(range(40))

# 2. Define Formatting Function
def formatting_func(examples):
    text = examples["text_prompt"]
    if isinstance(text, list):
        return text
    return [text]

# 3. Train
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,        # Now only 40 samples
    dataset_text_field = "text_prompt",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    formatting_func = formatting_func,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 300,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",

        # --- Early Stopping Configuration ---
        eval_strategy = "steps",
        eval_steps = 20,                # Check every 20 steps
        save_strategy = "steps",
        save_steps = 20,
        load_best_model_at_end = True,
        metric_for_best_model = "eval_loss",
        greater_is_better = False,
    ),
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)]
)

trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/36355 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/40 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 36,355 | Num Epochs = 1 | Total steps = 300
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176 of 7,655,986,688 (0.53% trained)
wandb: Currently logged in as: ankitdashcode (ankitdashcode-indian-institute-of-information-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
20,2.244800,2.147454
40,2.067200,2.059479
60,2.387300,2.036795
80,2.031300,2.011678
100,1.952100,2.000124
120,1.814000,1.990680
140,2.100700,1.983588
160,2.355500,1.972716
180,2.333000,1.967393
200,2.137100,1.963612


Unsloth: Not an error, but Qwen2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=300, training_loss=2.045600390036901, metrics={'train_runtime': 1252.7963, 'train_samples_per_second': 1.916, 'train_steps_per_second': 0.239, 'total_flos': 1.0277285300975616e+16, 'train_loss': 2.045600390036901, 'epoch': 0.06601386291121135})

In [7]:
# 7. Inference & Generation
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

print("Loading Test Data...")
test_df = load_data(test_dir)

# Prompt template WITHOUT the label (so the model fills it in)
inference_prompt = """You are an expert linguist. Analyze the following text for polarization.
Text: {}
Language: {}
Label:"""

predictions = []

print("Starting Inference...")
for index, row in test_df.iterrows():
    # Prepare input
    prompt = inference_prompt.format(row['text'], row['language'])
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate
    outputs = model.generate(
        **inputs,
        max_new_tokens = 10,
        use_cache = True
    )

    # Decode output
    decoded_output = tokenizer.batch_decode(outputs)[0]

    # Extract just the generated part (after "Label:")
    answer = decoded_output.split("Label:")[-1].strip()
    answer = answer.replace("<|END_OF_TURN_TOKEN|>", "").strip()

    # Map back to 0 or 1 (Integers)
    if "Not Polarized" in answer:
        pred_val = 0
    elif "Polarized" in answer:
        pred_val = 1
    else:
        # Fallback logic: if ambiguous, default to 0 (safe bet)
        pred_val = 0

    predictions.append(pred_val)
    if index % 10 == 0:
        print(f"Processed {index}/{len(test_df)}", end="\r")

# Store in temporary column 'prediction'
test_df['prediction'] = predictions
print("\nInference Complete.")

Loading Test Data...
Starting Inference...

Inference Complete.


In [8]:
# 8. Save Results in Submission Format
output_dir = './predictions_aya'
os.makedirs(output_dir, exist_ok=True)

languages = sorted(test_df['language'].unique())

# Identify the ID column explicitly
# Common variations in SemEval datasets
possible_id_cols = ['id', 'tweet_id', 'text_id', 'sample_id', 'unique_id']
id_col = next((c for c in test_df.columns if c.lower() in possible_id_cols), None)

if not id_col:
    print("WARNING: Could not find an ID column. Using default index.")

for lang in languages:
    # Filter for current language
    df_lang = test_df[test_df['language'] == lang].copy()

    # Select ONLY id and prediction, then rename prediction to polarization
    if id_col:
        out_df = df_lang[[id_col, 'prediction']].copy()
        # RENAME 'prediction' -> 'polarization'
        out_df.rename(columns={'prediction': 'polarization'}, inplace=True)
    else:
        # Fallback if no ID column
        out_df = df_lang[['prediction']].copy()
        out_df.rename(columns={'prediction': 'polarization'}, inplace=True)

    # Verify Columns are strictly: [id, polarization]
    # And verify values are 0 or 1
    print(f"Validating {lang}...")
    print(f"Columns: {list(out_df.columns)}")

    # Save as pred_[lang_code].csv
    # If lang is 'eng', file is 'pred_eng.csv'
    filename = f'pred_{lang}.csv'
    path = os.path.join(output_dir, filename)

    out_df.to_csv(path, index=False)
    print(f'Saved {filename} ({len(out_df)} rows)')

Validating amh...
Columns: ['id', 'polarization']
Saved pred_amh.csv (166 rows)
Validating arb...
Columns: ['id', 'polarization']
Saved pred_arb.csv (169 rows)
Validating deu...
Columns: ['id', 'polarization']
Saved pred_deu.csv (159 rows)
Validating eng...
Columns: ['id', 'polarization']
Saved pred_eng.csv (133 rows)
Validating fas...
Columns: ['id', 'polarization']
Saved pred_fas.csv (164 rows)
Validating hau...
Columns: ['id', 'polarization']
Saved pred_hau.csv (182 rows)
Validating hin...
Columns: ['id', 'polarization']
Saved pred_hin.csv (137 rows)
Validating ita...
Columns: ['id', 'polarization']
Saved pred_ita.csv (166 rows)
Validating nep...
Columns: ['id', 'polarization']
Saved pred_nep.csv (100 rows)
Validating spa...
Columns: ['id', 'polarization']
Saved pred_spa.csv (165 rows)
Validating tur...
Columns: ['id', 'polarization']
Saved pred_tur.csv (115 rows)
Validating urd...
Columns: ['id', 'polarization']
Saved pred_urd.csv (142 rows)
Validating zho...
Columns: ['id', 'polar